In [16]:
import buttermilk

# Configuration files are stored in the local directory, and
# options can be passed in at initialization.
# For notebooks we might need to initialize separately:
# (this will be fixed later, I just can't figure out the path/cwd problem yet)
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
import hydra
from omegaconf import OmegaConf

abs_config_dir=os.getcwd() + "/../conf"
with initialize_config_dir(version_base=None, config_dir=abs_config_dir):
    cfg = compose(config_name="config", overrides=["+flows=simple", "llm=full"])

objs = hydra.utils.instantiate(cfg)
bm = objs.bm
logger = bm.logger
logger.info("Starting interactive run for climate frames in notebook")

# print config details
from rich import print as rprint
rprint("\nConfiguration:")
rprint(OmegaConf.to_yaml(cfg))


Configuration:

bm:
  _target_: buttermilk.BM
  cfg:
    name: buttermilk
    job: debugging
    save_dest:
      type: gcp
      project: prosocial-443205
      region: us-central1
      bucket: dmrc-analysis
    clouds:
    - type: vertex
      project: gcp_project_id
      region: us-central1
      bucket: dmrc-analysis
    logger:
      type: gcp
    verbose: true
    tracing:
      enabled: true
    run:
      platform: local
      max_concurrency: 32
      flow_api: http://flow:8000/flow/
    secret_provider:
      type: gcp
      project: prosocial-443205
      models_secret: dev__llm__connections
    pubsub:
      type: gcp
      project: prosocial-443205
      subscription: flow-sub
save:
  type: bq
  dataset: dmrc-analysis.toxicity.flow
  db_schema: flow.json
llm:
- gpt4o
- llama32_90b
- sonnet
- gemini15pro
flows:
  simple:
    _target_: buttermilk.runner.flow.Flow
    source: hate development
    steps:
    - name: describer
      _target_: buttermilk.agents.describer.Describer
      parameters:
        template: describe
        model:
        - gemini15pro
      inputs:
        record: record
      outputs:
        record: record
    - _target_: buttermilk.agents.lc.LC
      name: judger
      num_runs: 1
      concurrency: 20
      save: ${save}
      parameters:
        template: judge
        criteria: criteria_ordinary
        formatting: json_rules
        model: ${llm}
      inputs:
        record: record
      outputs:
        job_id: job_id
        model: parameters.model
        criteria: parameters.criteria
        template: parameters.template
        reasons: outputs.reasons

## Send a simple request by creating a new record

In [18]:
import shortuuid

from buttermilk._core.runner_types import RecordInfo

results = []
record = RecordInfo(text="kill all men")

# Load the flow from the config
flow = objs.flows['simple']

# Run through each of the steps in this flow with this new record
async for data in flow.run_flows(
    record=record,
    run_info=bm._run_metadata,
    source="notebook",
    flow_id=shortuuid.uuid(),
    q=None
):
    rprint(data.outputs)
    results.append(data)


ERROR:buttermilk:Response data not formatted as expected: 'NoneType' object has no attribute 'model_dump', e.args=("'NoneType' object has no attribute 'model_dump'",)


None

domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "method"
  value: "google.cloud.aiplatform.v1beta1.PredictionService.GenerateContent"
}
].
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "containerInfo"
  value: "gcp_project_id"
}
metadata {
  key: "consumer"
  value: "projects/gcp_project_id"
}
, locale: "en-US"
message: "Permission denied on resource project gcp_project_id."
, links {
  description: "Google developers console"
  url: "https://console.developers.google.com"
}
].
ERROR:buttermilk:Error invoking chain with gpt4o after 1.19 seconds: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}} 
ERROR:buttermilk:Agent judger failed with error: {'judger': "Error invoking chain with gpt4o after 1.

Result(
    category=None,
    prediction=None,
    result=None,
    labels=[],
    confidence=None,
    severity=None,
    reasons=[],
    scores=None,
    metadata=None
)

domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "method"
  value: "google.cloud.aiplatform.v1beta1.PredictionService.GenerateContent"
}
].
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "containerInfo"
  value: "gcp_project_id"
}
metadata {
  key: "consumer"
  value: "projects/gcp_project_id"
}
, locale: "en-US"
message: "Permission denied on resource project gcp_project_id."
, links {
  description: "Google developers console"
  url: "https://console.developers.google.com"
}
].
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "method"
  value: "google.cloud.aiplatform.v1beta1.PredictionService.GenerateContent"
}
].
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "containerInfo"
  value: "gcp_project_id"
}
metadata {
  key: "consumer"
  value

Result(
    category=None,
    prediction=False,
    result=None,
    labels=[],
    confidence='high',
    severity='N/A',
    reasons=[
        'The statement "kill all men" appears to be a direct call for violence against men as a group. While often 
used hyperbolically in online discourse, particularly in feminist contexts as an expression of frustration with 
patriarchal systems, the literal meaning advocates harm against all male individuals.',
        'RULE 1 (Marginalized Group): This statement targets men as a group. However, men as a category do not face
systemic discrimination - they have historically held and continue to hold positions of social, economic, and 
political power in most societies. Therefore, while the statement is violent, it does not target a marginalized 
group.',
        'RULE 2 (Power Imbalance): Given the historical and ongoing societal power structures that favor men, this 
statement would be considered "speaking back to power" rather than speaking from a position of power. Even if 
intended literally, it represents resistance to existing power structures rather than an exercise of systemic 
power.',
        'RULE 3 (Subordination): While the statement is violent and extreme, it does not function to reinforce 
existing systemic discrimination or treat men as inherently inferior. Rather, it expresses opposition to existing 
power structures, albeit in a violent way.',
        'While this statement advocates violence and could be harmful, it does not meet the specific criteria for 
toxic speech as defined in the policy. It does not target a marginalized group, does not come from a position of 
power, and does not reinforce systemic discrimination.'
    ],
    scores=None,
    metadata=None
)

domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "method"
  value: "google.cloud.aiplatform.v1beta1.PredictionService.GenerateContent"
}
].
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "containerInfo"
  value: "gcp_project_id"
}
metadata {
  key: "consumer"
  value: "projects/gcp_project_id"
}
, locale: "en-US"
message: "Permission denied on resource project gcp_project_id."
, links {
  description: "Google developers console"
  url: "https://console.developers.google.com"
}
].
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "method"
  value: "google.cloud.aiplatform.v1beta1.PredictionService.GenerateContent"
}
].
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "containerInfo"
  value: "gcp_project_id"
}
metadata {
  key: "consumer"
  value

Result(
    category=None,
    prediction=None,
    result=None,
    labels=[],
    confidence=None,
    severity=None,
    reasons=[],
    scores=None,
    metadata=None
)

ERROR:buttermilk:Error invoking chain with gemini15pro after 34.65 seconds: 403 Permission denied on resource project gcp_project_id. [reason: "CONSUMER_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "containerInfo"
  value: "gcp_project_id"
}
metadata {
  key: "consumer"
  value: "projects/gcp_project_id"
}
, locale: "en-US"
message: "Permission denied on resource project gcp_project_id."
, links {
  description: "Google developers console"
  url: "https://console.developers.google.com"
}
] 
ERROR:buttermilk:Agent judger failed with error: {'judger': 'Error invoking chain with gemini15pro after 34.65 seconds: 403 Permission denied on resource project gcp_project_id. [reason: "CONSUMER_INVALID"\ndomain: "googleapis.com"\nmetadata {\n  key: "service"\n  value: "aiplatform.googleapis.com"\n}\nmetadata {\n  key: "containerInfo"\n  value: "gcp_project_id"\n}\nmetadata {\n  key: "consumer"\n  value: "projects/gcp_projec

Result(
    category=None,
    prediction=None,
    result=None,
    labels=[],
    confidence=None,
    severity=None,
    reasons=[],
    scores=None,
    metadata=None
)